# K-Nearest Neighbor Lab





In [7]:
import io
import math
import urllib.request

import pandas as pd
from numpy import add
from scipy.io import arff
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

## 1. Implement the k-nearest neighbor (KNN) algorithm

### Code requirements
- Use Euclidean distance to decide closest neighbors
- Implement both the regular (classifcation) version and the regression version
- Include optional distance weighting for both algorithms

In [8]:
def load_data(url: str):
    ftp_stream = urllib.request.urlopen(url)
    data, meta = arff.loadarff(io.StringIO(ftp_stream.read().decode('utf-8')))
    data_frame = pd.DataFrame(data)
    return data_frame

# Gets the euclidean distance between new point and all X rows for continuous data
def real_dist(X, newPoint, ax=1):
    x = X - newPoint
    s = (x.conj() * x).real
    sqrtFunction = np.vectorize(sqrt)
    return sqrtFunction(add.reduce(s, axis=ax, keepdims=False))

def sqrt(value):
    return math.sqrt(value)

def get_inv_dist_squared(distances):
    return 1.0 / (distances**2+0.0000001)


# Gets the top k rows from a matrix
def get_top_k(matrix, k):
    if len(matrix.shape) == 1:
        return matrix[:k]
    else:
        return matrix[:k, :]

# Gets a list of unique values for a 1d array
def get_unique(array):
    return np.unique(array)

# Gets the most frequent number from an array
def get_mode(array):
    vals, counts = np.unique(array, return_counts=True)
    mode_value = np.argwhere(counts == np.max(counts))
    return vals[mode_value].flatten()[0]

# Aggregates a 2d matrix by the group column, summing the agg column
def get_sum_aggregates(array, group_col_index, agg_col_index):
    unique_vals = get_unique(array[:, group_col_index])
    my_aggregation = np.array([[unique_val, array[array[:,group_col_index]==unique_val,agg_col_index].sum()] for unique_val in unique_vals], dtype='O')
    my_sorted_aggregation = my_aggregation[(-my_aggregation[:,-1]).argsort()]
    return my_sorted_aggregation

def dot_col_vecs(x1, x2):
    return np.dot(x1, x2)

def get_num_elements(array):
    product = 1
    for elem in array.shape:
        product *= elem
    return product

def decodeBytes(value):
    if type(value) == 'bytes':
        return value.decode()
    return value

def get_continuous_distance(inputFeaturePoint, predictionFeaturePoint):
    if inputFeaturePoint == '?' or predictionFeaturePoint == '?':
        return 1
    return inputFeaturePoint - predictionFeaturePoint

def get_nominal_distance(inputFeaturePoint, predictionFeaturePoint):
    if inputFeaturePoint == '?' or predictionFeaturePoint == '?':
        return 1
    return int(inputFeaturePoint != predictionFeaturePoint)

def normalize_data(array):
    newarray = array.copy()
    _, num_cols = array.shape
    for i in range(0, num_cols):
        curCol = array[:,i]
        curColMax = np.max(curCol)
        curColMin = np.min(curCol)
        def normalize(value):
            return (value - curColMin) / float(curColMax - curColMin)
        normalizeVectorized = np.vectorize(normalize)
        newarray[:,i] = normalizeVectorized(curCol)
    return newarray

def get_normalized_housing_data():
    vectorizedDecoder = np.vectorize(decodeBytes)
    training_data = vectorizedDecoder(np.array(load_data(r"https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/housing_train.arff")))
    train_x = training_data[:,:-1]
    train_y = training_data[:,-1]

    testing_data = vectorizedDecoder(np.array(load_data(r"https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/housing_test.arff")))
    test_x = testing_data[:,:-1]
    test_y = testing_data[:,-1]

    combined_data = np.concatenate((train_x, test_x))
    combined_normalized = normalize_data(combined_data)
    num_train_x_rows, _ = train_x.shape

    normalized_train_x = combined_normalized[:num_train_x_rows, :]
    normalized_test_x = combined_normalized[num_train_x_rows:, :]
    return normalized_train_x, train_y, normalized_test_x, test_y

def get_normalized_telescope_data():
    vectorizedDecoder = np.vectorize(decodeBytes)

    training_data = np.array(
        load_data(r"https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/magic_telescope_train.arff"))
    train_x = training_data[:, :-1]
    train_y = vectorizedDecoder(training_data[:, -1])

    testing_data = np.array(
        load_data(r"https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/magic_telescope_test.arff"))
    test_x = testing_data[:, :-1]
    test_y = vectorizedDecoder(testing_data[:, -1])

    combined_data = np.concatenate((train_x, test_x))
    combined_normalized = normalize_data(combined_data)
    num_train_x_rows, _ = train_x.shape

    normalized_train_x = combined_normalized[:num_train_x_rows, :]
    normalized_test_x = combined_normalized[num_train_x_rows:, :]
    return normalized_train_x, train_y, normalized_test_x, test_y

In [9]:
class KNNClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, columntype=[], weight_type='inverse_distance', regression=False, hasNominal=False):  ## add parameters here
        """
        Args:
            columntype for each column tells you if continues[real] or if nominal[categoritcal].
            weight_type: inverse_distance voting or if non distance weighting. Options = ["no_weight","inverse_distance"]
        """
        self.columntype = columntype  # Note This won't be needed until part 5
        self.weight_type = weight_type
        self.X = None
        self.y = None
        self.regression = regression
        self.has_nominal = hasNominal

    def fit(self, X, y):
        self.X = X
        self.y = y
        return self

    def predict(self, X, k_array):
        k_dictionary = {}
        for row in X:
            new_x, distances, new_y = self.get_distances(self.X, self.y, row)
            for k in k_array:
                new_x_copy = get_top_k(new_x, k)
                new_y_copy = get_top_k(new_y, k)
                distances_copy = get_top_k(distances, k)
                if self.regression:
                    if self.weight_type == 'inverse_distance':
                        inv_distances = get_inv_dist_squared(distances_copy)
                        numerator = dot_col_vecs(inv_distances, new_y_copy)
                        denominator = inv_distances.sum()
                        guess = float(numerator)/denominator
                        self.add_k_guess(k_dictionary, guess, k)
                    else:
                        guess = new_y_copy.sum()/float(get_num_elements(new_y_copy))
                        self.add_k_guess(k_dictionary, guess, k)
                else:
                    if self.weight_type == 'inverse_distance':
                        inv_distances = get_inv_dist_squared(distances_copy)
                        aggregates = get_sum_aggregates(np.append(inv_distances[..., None], new_y_copy[..., None], axis=1), 1, 0)
                        guess = aggregates[0, 0]
                        self.add_k_guess(k_dictionary, guess, k)
                    else:
                        guess = get_mode(new_y_copy)
                        self.add_k_guess(k_dictionary, guess, k)

        return k_dictionary

    # Returns the Mean score given input data and labels
    def score(self, X, y, k_array):
        """ Return accuracy of model on a given dataset. Must implement own score function.
        Args:
            X (array-like): A 2D numpy array with data, excluding targets
            y (array-like): A 2D numpy array with targets
        Returns:
            score : float
                Mean accuracy of self.predict(X) wrt. y.
        """
        k_dictionary = self.predict(X, k_array)
        finalScores = []
        for k in k_array:
            results = k_dictionary[k]
            if self.regression:
                mse = ((y - results)**2).sum() / float(len(y))
                finalScores.append(mse)
            else:
                correct = 0
                for i in range(0, len(results)):
                    if results[i] == y[i]:
                        correct += 1
                finalScores.append(float(correct) / len(results))
        return finalScores

    def get_prediction_by_count(self, y):
        return get_mode(y)

    def add_k_guess(self, k_dictionary, guess, k):
        if k not in k_dictionary:
            k_dictionary[k] = []
        k_dictionary[k].append(guess)

    # Returns the X and y values sorted by distance from the new point along with their corresponding distances
    def get_distances(self, X, y, newPoint):
        if not self.has_nominal:
            distances = real_dist(X, newPoint)
            augmented_with_y = np.append(X, np.column_stack([y]), axis=1)
            augmented_with_dist = np.append(augmented_with_y, np.column_stack([distances]), axis=1)
            _, num_cols = augmented_with_dist.shape
            sorted_augmented = augmented_with_dist[augmented_with_dist[:, num_cols - 1].argsort()]
            new_x = sorted_augmented[:, :-2]
            new_y = sorted_augmented[:, -1]
            dist = sorted_augmented[:, -2]
            return new_x, new_y, dist
        else:
            all_distances = []
            for i, row in enumerate(X):
                row_distances = []
                for j, col in enumerate(row):
                    if self.columntype[j] == 'nominal':
                        row_distances.append(get_nominal_distance(X[i, j], newPoint[j]))
                    else:
                        row_distances.append(get_continuous_distance(X[i, j], newPoint[j]))
                row_distances = np.array(row_distances)
                final_distance = np.sqrt(np.power(row_distances,2).sum())
                all_distances.append(final_distance)
            distances = np.array(all_distances)
            augmented_with_y = np.append(X, np.column_stack([y]), axis=1)
            augmented_with_dist = np.append(augmented_with_y, np.column_stack([distances]), axis=1)
            _, num_cols = augmented_with_dist.shape
            sorted_augmented = augmented_with_dist[augmented_with_dist[:, num_cols - 1].argsort()]
            new_x = sorted_augmented[:, :-2]
            new_y = sorted_augmented[:, -1]
            dist = sorted_augmented[:, -2]
            return new_x, new_y, dist

## Debug and Evaluation

Debug and Evaluate your model using the parameters below:
- Use distance weighting
- KNN = 3 (three nearest neighbors)
- Don’t normalize the data
- Use Euclidean Distance
---

### 1.1 (20%) Debug using this [training set](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/glass_train.arff) and this [test set](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/glass_test.arff)

Expected Results:
- Not using inverse weighted distancing = roughly [68.29%]
- Link to [debug solution](https://github.com/cs472ta/CS472/blob/master/debug_solutions/glass_no_inv_predictions.txt)
- Using inverse weighted distancing = roughly [74.39%]
- Link to [debug solution](https://github.com/cs472ta/CS472/blob/master/debug_solutions/glass_inv_predictions.txt)

In [11]:
def do_debug():
    vectorizedDecoder = np.vectorize(decodeBytes)

    training_data = np.array(load_data(r"https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/glass_train.arff"))
    train_x = training_data[:,:-1]
    train_y = vectorizedDecoder(training_data[:,-1])

    testing_data = np.array(load_data(r"https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/glass_test.arff"))
    test_x = testing_data[:,:-1]
    test_y = vectorizedDecoder(testing_data[:,-1])

    knn = KNNClassifier(weight_type="")
    knn.fit(train_x, train_y)
    print(knn.score(test_x, test_y, [3]))

    knn2 = KNNClassifier()
    knn2.fit(train_x, train_y)
    print(knn2.score(test_x, test_y, [3]))
do_debug()

[0.6829268292682927]
[0.7439024390243902]


### 1.2 (20%) Evaluate

We will evaluate your model based on its performance on the [diabetes](https://archive.ics.uci.edu/ml/datasets/Diabetes) problem.
- Use this [training set](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/diabetes_train.arff) and this [test set](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/diabetes_test.arff) and have your code print the accuracy.

In [10]:
def do_eval():
    vectorizedDecoder = np.vectorize(decodeBytes)

    training_data = np.array(load_data(r"https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/diabetes_train.arff"))
    train_x = training_data[:,:-1]
    train_y = vectorizedDecoder(training_data[:,-1])

    testing_data = np.array(load_data(r"https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/diabetes_test.arff"))
    test_x = testing_data[:,:-1]
    test_y = vectorizedDecoder(testing_data[:,-1])

    knn = KNNClassifier(weight_type="")
    knn.fit(train_x, train_y)
    print(knn.score(test_x, test_y, [3]))

    knn2 = KNNClassifier()
    knn2.fit(train_x, train_y)
    print(knn2.score(test_x, test_y, [3]))
do_eval()

[0.8411458333333334]
[0.890625]


## 2. KNN with and without normalization

- Use the [magic telescope](http://archive.ics.uci.edu/ml/datasets/MAGIC+Gamma+Telescope) task with this [training set](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/magic_telescope_train.arff) and this [test set](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/magic_telescope_test.arff) 

### 2.1 (5%)
- Try it with k=3 and without distance weighting and *without* normalization


In [12]:
def do_magic_without_normalization():
    vectorizedDecoder = np.vectorize(decodeBytes)

    training_data = np.array(load_data(r"https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/magic_telescope_train.arff"))
    train_x = training_data[:,:-1]
    train_y = vectorizedDecoder(training_data[:,-1])

    testing_data = np.array(load_data(r"https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/magic_telescope_test.arff"))
    test_x = testing_data[:,:-1]
    test_y = vectorizedDecoder(testing_data[:,-1])

    knn = KNNClassifier(weight_type="")
    knn.fit(train_x, train_y)
    print(knn.score(test_x, test_y, [3]))
do_magic_without_normalization()

[0.8082808280828083]


### 2.2 (5%)
- Try it with k=3 without distance weighting and *with* normalization (input features normalized between 0 and 1). Use the normalization formula (x-xmin)/(xmax-xmin)

In [13]:
def do_magic_with_normalization():
    vectorizedDecoder = np.vectorize(decodeBytes)

    training_data = np.array(load_data(r"https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/magic_telescope_train.arff"))
    train_x = training_data[:,:-1]
    train_y = vectorizedDecoder(training_data[:,-1])

    testing_data = np.array(load_data(r"https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/magic_telescope_test.arff"))
    test_x = testing_data[:,:-1]
    test_y = vectorizedDecoder(testing_data[:,-1])

    combined_data = np.concatenate((train_x,test_x))
    combined_normalized = normalize_data(combined_data)
    num_train_x_rows, _ = train_x.shape

    normalized_train_x = combined_normalized[:num_train_x_rows, :]
    normalized_test_x = combined_normalized[num_train_x_rows:, :]

    knn2 = KNNClassifier(weight_type="")
    knn2.fit(normalized_train_x, train_y)
    print(knn2.score(normalized_test_x, test_y, [3]))
do_magic_with_normalization()

[0.8304830483048304]


*Discuss the results of using normalized data vs. unnormalized data*
The model trained on the normalized data was a bit more accurate than the model trained on the unnormalized data. After predicting the test set with k=3, I got an accuracy of approximately 80% using the unnormalized data and an accuracy of 83% using the normalized data. This makes sense, as attributes that naturally have a larger range generally have higher distance metrics whereas if the data is normalized, each feature gets an opportunity to provide input as the distance metrics are now standardized. To normalize the data, I combined both the training and test datasets first so that the range across which they were normalized was equivalent. One thing that could be worth further investigation is the scale across which the data is normalized. For example if the data was not necessarily linear, it could be better to find another normalizing function that better distributed the data to give the model a better opportunity to learn.

### 2.3 (5%)

- Using your normalized data, create one graph with classification accuracy on the test set on the y-axis and k values on the x-axis. 
    - Use odd values of k from 1 to 15.
- As a rough sanity check, typical knn accuracies for the magic telescope data set are 75-85%

In [14]:
def graph_magic():
    vectorizedDecoder = np.vectorize(decodeBytes)

    training_data = np.array(load_data(r"https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/magic_telescope_train.arff"))
    train_x = training_data[:,:-1]
    train_y = vectorizedDecoder(training_data[:,-1])

    testing_data = np.array(load_data(r"https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/magic_telescope_test.arff"))
    test_x = testing_data[:,:-1]
    test_y = vectorizedDecoder(testing_data[:,-1])

    combined_data = np.concatenate((train_x,test_x))
    combined_normalized = normalize_data(combined_data)
    num_train_x_rows, _ = train_x.shape

    normalized_train_x = combined_normalized[:num_train_x_rows, :]
    normalized_test_x = combined_normalized[num_train_x_rows:, :]

    knn2 = KNNClassifier(weight_type="")
    knn2.fit(normalized_train_x, train_y)
    k_array = [1,3,5,7,9,11,13,15]
    accuracy = knn2.score(normalized_test_x, test_y, [1,3,5,7,9,11,13,15])
    print(accuracy)

    plt.plot(k_array, accuracy)
    plt.title("Classification Accuracy by k-value")
    plt.xlabel("K-value")
    plt.ylabel("Accuracy")
    plt.show()
graph_magic()

When I ran knn on the normalized data and graphed it for different k-values, I was somewhat surprised at how quickly the accuracy leveled off as k-values increased, flattening out at around k=5. This makes sense, because with k-values smaller than 5, noise can have a much larger impact on the data. If you happened to predict a point close to a couple of noisy points, you would predict the wrong class. However, with k=5, it would be much less likely that you would find 5 noisy points in the same region. I continued to increase the k-values (although not pictured) the classification accuracy would go down. This also makes sense, because eventually you would be looking at neighbors in different "clusters" or with very different attributes, and they would get a chance to vote. This would eventually just classify everything as the majority if you weren't using a weighted distance metric.

*For the rest of the experiments use only normalized data*

## 3. (10%) KNN regression

- Use the regression variation of your algorithm (without distance weighting) on the [housing price prediction](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html) problem.  Use this [training set](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/housing_train.arff) and this [test set](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/housing_test.arff). Note this data set has an example of an inappropriate use of data which we will discuss.
- Use Mean Square Error (MSE) on the test set as your accuracy metric for this case
    - Do not normalize regression output values
- Graph MSE on the test set with odd values of k from 1 to 15

In [15]:
def do_housing():
    normalized_train_x, train_y, normalized_test_x, test_y = get_normalized_housing_data()
    knn = KNNClassifier(weight_type='',regression=True)
    knn.fit(normalized_train_x, train_y)
    k_array = [1,3,5,7,9,11,13,15]
    mses = knn.score(normalized_test_x, test_y, k_array)
    print(mses)

    plt.plot(k_array, mses)
    plt.title("Housing MSE by k-value")
    plt.xlabel("K-value")
    plt.ylabel("MSE")
    plt.show()
do_housing()

When examining the results of the MSE graph, I found it somewhat interesting that the knn model with the lowest MSE used k=5. This follows a similar pattern to the classification accuracy of the telescope dataset as the average/choice of a few closer points gives a better score than only a few possibly noisy points or a set of points so large that doesn't accurately capture the information within a certain feature region. I also noticed the MSE seemed to level out at about k=13. I imagine that if we used a weighted distance metric, the resulting graph would not be as jagged, but have somewhat more similar MSE values as the k-values increased, although the MSE might slightly get worse as the k-value increases.

## 4. KNN with distance weighting
- Repeat your experiments for magic telescope and housing using distance-weighted (inverse of distance squared) voting and discuss your results.

### 4.1 (7.5%) Magic Telescope Dataset

In [21]:
def do_magic_distance_weighting():
    normalized_train_x, train_y, normalized_test_x, test_y = get_normalized_telescope_data()
    knn = KNNClassifier()
    knn.fit(normalized_train_x, train_y)
    k_array = [1,3,5,7,9,11,13,15]
    accuracy = knn.score(normalized_test_x, test_y, k_array)
    print(accuracy)

    plt.plot(k_array, accuracy)
    plt.title("Classification Accuracy by k-value (with weighted metric)")
    plt.xlabel("K-value")
    plt.ylabel("Accuracy")
    plt.show()
do_magic_distance_weighting()

### 4.2 (7.5%) Housing Dataset

In [23]:
def do_housing_distance_weighting():
    normalized_train_x, train_y, normalized_test_x, test_y = get_normalized_housing_data()
    knn = KNNClassifier(regression=True)
    knn.fit(normalized_train_x, train_y)
    k_array = [1,3,5,7,9,11,13,15]
    mses = knn.score(normalized_test_x, test_y, k_array)
    print(mses)

    plt.plot(k_array, mses)
    plt.title("Housing MSE by k-value (with weighted metric)")
    plt.xlabel("K-value")
    plt.ylabel("MSE")
    plt.show()
do_housing_distance_weighting()

*Discuss your results*
Using distance weighting slightly improved the accuracy of the Magic Telescope Dataset classification and the MSE of the housing dataset. The Telescope classification accuracy was very similar and followed a similar shape to that of classification without the weighted inverse square distance metric. The classification accuracy was also less volatile when higher k-values were reached. This makes sense as points get farther and farther away from the original point they have less of a weight and can't sway the selection (vote) very much whereas without the weighted distance metric each point had an exactly equal impact on the selection, so points located farther away could determine the weight of a single point simply by majority no matter what its feature values were if the k-value was high enough.

I thought that it was fascinating when I saw the graph of the Housing dataset MSE by k-value because it followed the prediction I made earlier. The MSE stabilized instead of wildly oscillating. When the k-value increases and more points are used that are farther away, they don't have such a heavy impact on the MSE as they aren't weighted very much. Eventually we would expect the MSE to get worse as we start including too many points in the classification that are dissimilar from the original point. At around k=7 the best MSE was reached which was a larger value than the optimal for the non-distance-weighted model.


## 5. (10%) KNN with nominal and unknown data

- Use the [credit-approval](https://archive.ics.uci.edu/ml/datasets/Credit+Approval) task and this [dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/credit_approval.arff)
    - Use a 70/30 split of the data for the training/test set
- Note that this set has both continuous and nominal attributes, together with don’t know values. 
- Implement and justify a distance metric which supports continuous, nominal, and don’t know attribute values
    - You need to handle don't knows with the distance metric, not by imputing a value.
    - More information on distance metrics can be found [here](https://www.jair.org/index.php/jair/article/view/10182/24168).
- Use your own choice for k.
- As a rough sanity check, typical knn accuracies for the credit data set are 70-80%.

In [24]:
def do_credit():
    data = np.array(pd.read_csv(r"https://raw.githubusercontent.com/maclaurin36/CS312-knn/master/crx.csv"))
    nominal_feature_indices = [0,3,4,5,6,8,9,11,12]
    continuous_feature_indices = [1,2,7,10,13,14]
    coltypes = ['nominal','continuous','continuous','nominal','nominal','nominal','nominal','continuous','nominal','nominal','continuous','nominal','nominal','continuous','continuous']
    for colIndex in continuous_feature_indices:
        normalize_col = data[:, colIndex]
        no_missing = normalize_col[normalize_col!='?'].astype(float)
        max = np.max(no_missing)
        min = np.min(no_missing)
        for i, val in enumerate(normalize_col):
            if val == '?':
                data[i,colIndex] = '?'
            else:
                data[i,colIndex] = (float(normalize_col[i]) - min) / float(max - min)

    train_x, test_x, train_y, test_y = train_test_split(data[:,:-1], data[:,-1], test_size=0.3)
    knn = KNNClassifier(hasNominal=True, columntype=coltypes)
    knn.fit(train_x, train_y)
    print(knn.score(test_x, test_y, [3,5,7]))
do_credit()

[0.8454106280193237, 0.8599033816425121, 0.8599033816425121]


*Explain and justify your distance metric and discuss your results*
My distance metric was that the normalized distance between attributes was 1 if the attribute was unknown. This led the model to consider those data points as farther away. This is useful as there may be an underlying reason that the data was not collected or known. This underlying reason could lead to a different neighborhood of points, so it is likely best to just ignore the missing attributes.
For the nominal attributes, I used a simple distance metric of 1 if they didn't match and 0 if they did match. I did this because without more in-depth knowledge of the categories I couldn't determine if some features were generally more closely related or not. This metric was also much easier to compute than comparing the output classes of the training data for a given class and seeing which ones were most similar. I did however consider implementing that model and think it could be a valuable approach to handling unknown values in the credit data set.
For the continuous attributes I simply used the euclidean distance as was used in the previous exercises with unknowns handled as simply a distance of 1 from any point.

After testing knn models with k= 3, 4, and 5 on the credit dataset with the above piecewise distance metric, I generally got an average accuracy in the lower to mid 80s, varying according to the training test split with occasional results in the 70s. Given these results, I believe that the distance metric I used was a good way to classify this dataset.

## 6. (10%) Scikit-Learn KNN 
- Use the scikit-learn KNN version on magic telescope and housing and compare your results
- Try out different hyperparameters to see how well you can do. 

In [30]:
def run_scikit():
    normalized_train_x, train_y, normalized_test_x, test_y = get_normalized_telescope_data()

    print("Classification with telescope")
    knnClassifier = KNeighborsClassifier(5)
    knnClassifier.fit(normalized_train_x, train_y)
    print("k=5")
    print(knnClassifier.score(normalized_test_x, test_y))

    knnClassifier = KNeighborsClassifier(n_neighbors=10)
    knnClassifier.fit(normalized_train_x, train_y)
    print("k=10")
    print(knnClassifier.score(normalized_test_x, test_y))

    knnClassifier = KNeighborsClassifier(weights='distance')
    knnClassifier.fit(normalized_train_x, train_y)
    print("k=5, distance weightings")
    print(knnClassifier.score(normalized_test_x, test_y))

    knnClassifier = KNeighborsClassifier(weights='distance', n_neighbors=10)
    knnClassifier.fit(normalized_train_x, train_y)
    print("k=10, distance weightings")
    print(knnClassifier.score(normalized_test_x, test_y))

    print()
    print("Regression with Housing data")
    normalized_train_x, train_y, normalized_test_x, test_y = get_normalized_housing_data()
    knnRegressor = KNeighborsRegressor()
    knnRegressor.fit(normalized_train_x, train_y)
    print("k=5")
    print(knnRegressor.score(normalized_test_x, test_y))

    knnRegressor = KNeighborsRegressor(n_neighbors=10)
    knnRegressor.fit(normalized_train_x, train_y)
    print("k=10")
    print(knnRegressor.score(normalized_test_x, test_y))

    knnRegressor = KNeighborsRegressor(n_neighbors=7, weights='distance')
    knnRegressor.fit(normalized_train_x, train_y)
    print("k=7, distance weightings")
    print(knnRegressor.score(normalized_test_x, test_y))

    knnRegressor = KNeighborsRegressor(weights='distance')
    knnRegressor.fit(normalized_train_x, train_y)
    print("k=10, distance weightings")
    print(knnRegressor.score(normalized_test_x, test_y))
run_scikit()

Classification with telescope
k=5
0.8432343234323433
k=10
0.8402340234023402
k=5, distance weightings
0.8460846084608461
k=10, distance weightings
0.8492349234923492

Regression with Housing data
k=5
0.7957406361552128
k=10
0.7081491553203445
k=5, distance weightings
0.8356040739475871
k=10, distance weightings
0.8441169760654081


*Report your comparison*
Before I actually report the comparison of how well my models did when compared to scikit learns, I want to note that it amazes me how INCREDIBLY fast scikit learns models run in comparison to mine. I tried to use numpy operations to implement my model, but even so, the magic telescope dataset takes about 3ish minutes to run. Scikit learn's models take only a few seconds.

When comparing the results of the scikit learns models to my own for the telescope dataset, I found very little differences between the classification accuracies. Each model scored roughly between 84-85%. I tried adjusting the k-values and the weight metrics, and found that similar to my own model using inverse distance squared weightings and a k value between 5-10 yielded the highest accuracies.

When comparing the results of the housing models with my own, I found that generally k-values in the lower teens did best just as well in scikit as they did in my own implementation. If we look at the graph I generated when running my model, we see that the minimum MSE was achieved at about k = 7. This is almost equivalent to scikit learn's results. There was also a decent amount of variability in the scoring of the regressor with the housing dataset with different parameter values, although this difference was removed/minimized when distance weightings were used.

## 7. (optional 5% extra credit): Reducing the data set
- Choose either of the data sets above and use the best k value you found.
- Implement a reduction algorithm that removes data points in some rational way such that performance does not drop too drastically on the test set given the reduced training set.
- Compare your performance on the test set for the reduced and non-reduced versions and give the number (and percentage) of training examples removed from the original training set. 
    - Note that performance for magic telescope is classification accuracy and for housing it is mean squared error.
    - Magic Telescope has about 12,000 instances and if you use a leave one out style of testing for your data set reduction, then your algorithm will run slow since that is n^2 at each step.
        - If you wish, you may use a random subset of 2,000 of the magic telescope instances.
    - More information on reduction techniques can be found [here](http://axon.cs.byu.edu/~martinez/classes/478/slides/IBL.pdf).

In [20]:
# Code here

Discussion. How well did it do?